In [1]:
from utils.SystemMonitor import SystemMonitor
from utils.Table import Table

import zipfile
import os
import requests
import time
import tensorflow as tf
from importlib import import_module

# Dammit sci-kit
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

2023-11-22 11:32:30.113310: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 11:32:33.739218: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 11:32:33.744892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 11:32:41.050889: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Might want to drop notebook for sampling because of jupyter and kernel overhead

In [2]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download&confirm=1"

    session = requests.Session()

    response = session.get(URL, params={"id": id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"id": id, "confirm": token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

In [6]:
# Check dataset
dataset_folder = "dataset"

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

# Retrieve the list of folders inside the dataset folder that start with 'n'
folders = [folder for folder in os.listdir(dataset_folder) if folder.startswith('n') and os.path.isdir(os.path.join(dataset_folder, folder))]

# Unzip if list is empty
if len(folders) != 100:
    # Assuming you have the dataset ZIP file (dataset.zip) in the current directory
    if os.path.exists("dataset/Imagenet100-test-samples.zip"):
        with zipfile.ZipFile("dataset/Imagenet100-test-samples.zip", 'r') as zip_ref:
            zip_ref.extractall(dataset_folder)
        print("Dataset has been unzipped.")
    else:
        print("No dataset available to unzip. Downloading...")
        # https://drive.google.com/file/d/16zv4JbOPBVq0VQRXPAtosh8WE8GLbGoy/view?usp=sharing
        # https://drive.google.com/u/0/uc?id=16zv4JbOPBVq0VQRXPAtosh8WE8GLbGoy&export=download
        
        # URL of the file to download
        file_url = 'https://doc-04-2k-docs.googleusercontent.com/docs/securesc/22qmp18qg50lnev40v0qet1opj39hdlp/50dhcve33t6bs7scqpb7cpgs96tgijeq/1698506700000/12872051526394491434/12872051526394491434/16zv4JbOPBVq0VQRXPAtosh8WE8GLbGoy?e=download&ax=AI0foUoSOiQsVP1Yyg4kI4Z37psW41jAfaBnVqvovryyUP7_zeqOwn3VS_8W0f4oi84bVtgh509msfwmEkqNMzurjcyIn4ybJfcgaGpWO7KLv8HksKxYRx8EhdGSqUZ0JEQX5UKSI2oA_GgnEbs-EokO3z11RBM4-cDy4BcGcSaLYjvRA1AOO-Em7u-kBHQr85z27YgV7AXhERFo_PgbDutqAizF05ADjBKgrL9ZJyHtU-5sF8SGDrRykyKCpa7LW-avhUjVD-WCScfYw6VshegN6lMrTVCDEfMTj2IPM3LrOuLILSPryI5iTcTBVpkdiFy-cgHhS4d86ZnHRmdkvC_Y8bTxTWsDkEatDoUsdF3sLHlgGEbXAKADGxFmu_9t_dyBvuefKl3au9phUIdQeR2DLgEvd2g4F9NkGOd8cEnx4LfOdX2gv9-cMzgZrgbQo1RNXrY5TzKv5TTDe1rhl1lXgzU4ga0pMvN8VITi16ycWQ12yH6-psuPl9J3HPvUeWjGNvhm5OGnwjUYQS7SXkg_HcRnzvvyApvapFoVRe-F8oVT2meeqEnWWVGPY-ZAPwo-f-3Fv8Y-tOXzF7Qn_VjvsjwPaSFEdXDxOHEJ1szUHkHLCQXhLRyOYWrydVl251ATk4TxvE5aQSjPs1nE0c1q3ffvdUdL7YLGH8N6op1Qtye5xn180892DQNgcYUk9VxaJkToQQuTD-Vaj8aPXNQQuFTieEK18aASc5pcxXJUpH1JJP_GQBbiphWXOpry5r1MoNjvfBz0Iau71KgqNCObb2LdLrRt3vRRjUBRgPt9GPvqa6IqbyWMzQ2RvYP64k7IqZwbyQf4fKmw5qQKKSUGnGPhlouB0u7zaft7qknVV4An9l-3oBw8QtJ80Wj9yQAt2g&uuid=16afc191-3c70-4bb8-ba0f-bab52f256c8f&authuser=0&nonce=2ot0f4ccnm12i&user=12872051526394491434&hash=5c8mp3j5h0ev47le7cpqkgkmmjctbuo3'

        # Call the function to download the file
        download_file_from_google_drive("16zv4JbOPBVq0VQRXPAtosh8WE8GLbGoy", "dataset/Imagenet100-test-samples.zip")

        if download_file_from_google_drive:
            time.sleep(5)
            if os.path.exists("dataset/Imagenet100-test-samples.zip"):
                with zipfile.ZipFile("dataset/Imagenet100-test-samples.zip", 'r') as zip_ref:
                    zip_ref.extractall(dataset_folder)
                print("Dataset has been unzipped.")
else:
    print("Dataset is already available.")


Dataset has been unzipped.


In [ ]:
models_folder = "models"
file_name = "Samples6.csv"

# ONLY RUNS ON NEW SAMPLE GROUP
if not os.path.exists("dataframes/" + file_name):
    Table.create_empty_df(file_name, ["TimeStamp", "Name", "CPU_CORES", "CPU_CORES_LOG", "CPU_FREQ", "RAM_TOTAL_GB", "RAM_AV_GB", "RAM_USED_GB", "SWAP_TOTAL_GB", "SWAP_USED_GB", "CPU%", "RAM%", "RC", "WC", "RB", "WB", "dt", "accuracy", "precision", "recall", "f1"])
else:
    # raise(FileExistsError)
    print("Sample file already created. Did you want to start a new batch?")

Sample file already created. Did you want to start a new batch?


In [ ]:
# Create a TensorFlow session
config = tf.compat.v1.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
config.graph_options.rewrite_options.dependency_optimization = -1

# Set reduce_retracing to True or False
config.graph_options.rewrite_options.auto_mixed_precision = True

# Create a session with the specified configuration
sess = tf.compat.v1.Session(config=config)

Device mapping: no known devices.


# Target Models Debug

In [ ]:
def TestModel(m_name: str):
    # Get the model from the files
    model_file = [f[:-3] for f in os.listdir(models_folder) if f.endswith(".py") and f == m_name+".py"][0]

    try:
        # Import the module dynamically
        model_module = import_module(f"{models_folder}.{model_file}")
        
        # Get the class name
        class_name = model_file.replace("Ss", "SS")
            
        # Create an instance of the model class
        model_instance = None
        _ctr = 0
        while True:
            # Some models give random errors so they must be run multiple times....
            _no_rand_error = True 

            # Handle model instantiation errors
            try:
                model_instance = getattr(model_module, class_name)()
            # except AttributeError as e: # handle random error
            #     _no_rand_error = False
            #     _ctr += 1
            #     print("Execution failed on " + class_name + "... try " + str(_ctr))
            except Exception as e: # handle actual errors
                raise e

            if _no_rand_error:
                break
        
        if model_instance == None:
            raise ReferenceError

        # Instantiate a new system resource monitor
        monitor = SystemMonitor()
        
        # Start monitoring
        monitor.start(interval=0.1)
        # Make predictions
        model_instance.predict()
        # Stop monitoring
        monitor.stop()
        
        # Append data to the samples file for each model
        Table.append_data_to_samples(file_name, model_instance.__class__.__name__, monitor, model_instance)

    except Exception as e:
        raise(e)

In [ ]:
# # Take lots of samples of ResNet50 for quantitative assignment
# for _ in range(0,50):
#     TestModel("SS_ResNet50")

Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 0s 166ms/step
Monitoring stopped.
1 row(s) appended to Samples3.csv.
Appended to sample file:
['23:10:31:14:41:40', 'SS_ResNet50', 1, 1, 2808.002, '17.19', '15.11', '1.71', '0.95', '0.00', 66.15714285714287, 12.280000000000005, 2.761904761904762, 0.6190476190476191, 258633.14285714287, 4056.9904761904763, 10.499999999999979, '0.45', '0.9666666666666666', '0.45', '0.46749999999999997']
Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 0s 167ms/step
Monitoring stopped.
1 row(s) appended to Samples3.csv.
Appended to sample file:
['23:10:31:14:41:53', 'SS_ResNet50', 1, 1, 2808.002, '17.19', '15.08', '1.73', '0.95', '0.00', 68.09361702127657, 12.298404255319161, 1.7553191489361701, 0.7393617021276596, 178960.3404255319, 4836.765957446809, 8.299999999999986, '0.8', '1.0', '0.8', '0.8595238095238095']
Monitoring started with an interval of 0.1 seconds.
1/1 [=====

In [ ]:
# Stop execution for convenience
# raise SystemExit

SystemExit: 

/home/ubuntu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# All Models at once

In [ ]:
# Get a list of all Python files in the folder (excluding __init__.py and SS_model.py)
model_files = [f[:-3] for f in os.listdir(models_folder) if f.endswith(".py") and f != "__init__.py" and f != "SS_model.py"]

# Ease of use print nr of found models
print("Found " + str(len(model_files)) + " models.")
for file in model_files:
    print(file)

# Sanity time remaining
est_time_remaining = {"d": 0, "H": 0, "m": 0, "s": 0}
total_iter = 0
total_iter_done = 0
est_time_per_iter = 0
avg_time_per_iter_v = []

def compute_est_time_remaining(monitor: SystemMonitor):
    avg_time_per_iter_v.append(monitor.totalTime)

    # Only keep 38 values
    if len(avg_time_per_iter_v) > 38:
        avg_time_per_iter_v.pop(0)

    # Average last times
    est_time_per_iter = sum(avg_time_per_iter_v) / len(avg_time_per_iter_v)
    
    # Compute in seconds
    est_time_remaining_in_seconds = est_time_per_iter * (total_iter - total_iter_done)

    # Update struct
    est_time_remaining["d"] = est_time_remaining_in_seconds // (24 * 3600)
    est_time_remaining_in_seconds = est_time_remaining_in_seconds % (24 * 3600)
    est_time_remaining["H"] = est_time_remaining_in_seconds // 3600
    est_time_remaining_in_seconds %= 3600
    est_time_remaining["m"] = est_time_remaining_in_seconds // 60
    est_time_remaining_in_seconds %= 60
    est_time_remaining["s"] = est_time_remaining_in_seconds

# Nr iterations per batch let's say 10 * 10 random batches = 100
nr_iterations = 10

# Compute iterations
total_iter = nr_iterations * len(model_files)

# Loop iterations so we get a good dataset for each system resource
for _ in range(0, nr_iterations):
    # Count them for debug
    nr_models_executed = 0

    # Iterate through the model files and import the classes
    for model_file in model_files:
        print("Starting execution of " + str(model_file) + "... model nr." + str(nr_models_executed) + " out of " + str(len(model_files)) + " models.")
        try:
            # Import the module dynamically
            model_module = import_module(f"{models_folder}.{model_file}")
            
            # Get the class name
            class_name = model_file.replace("Ss", "SS")
                
            # Create an instance of the model class
            model_instance = None
            _ctr = 0
            while True:
                # Some models give random errors so they must be run multiple times....
                _no_rand_error = True 

                # Handle model instantiation errors
                try:
                    model_instance = getattr(model_module, class_name)()
                except AttributeError as e: # handle random error
                    _no_rand_error = False
                    _ctr += 1
                    print("Execution failed on " + class_name + "... try " + str(_ctr))
                except Exception as e: # handle actual errors
                    raise e

                if _no_rand_error:
                    break
            
            if model_instance == None:
                raise ReferenceError

            # Instantiate a new system resource monitor
            monitor = SystemMonitor()
            
            # Start monitoring
            monitor.start(interval=0.1)
            # Make predictions
            model_instance.predict()
            # Stop monitoring
            monitor.stop()
            
            # Append data to the samples file for each model
            Table.append_data_to_samples(file_name, model_instance.__class__.__name__, monitor, model_instance)

            # Increment counter
            nr_models_executed += 1
            total_iter_done += 1

            # Compute remaining time
            compute_est_time_remaining(monitor)

            # Print which model we are on
            print("Measured " + str(total_iter_done) + " out of " + str(total_iter) + " samples.")
            print("Estimated time remaining: " + str(est_time_remaining))
        except Exception as e:
            print("Executed " + str(nr_models_executed) + " out of " + str(len(model_files)) + " models.")
            raise(e)

Found 38 models.
SS_EfficientNetV2B2
SS_ConvNeXtBase
SS_EfficientNetV2B1
SS_EfficientNetV2B0
SS_ResNet101
SS_EfficientNetB1
SS_ConvNeXtXLarge
SS_VGG19
SS_EfficientNetB5
SS_EfficientNetB2
SS_MobileNet
SS_ResNet50V2
SS_EfficientNetB4
SS_ConvNeXtTiny
SS_InceptionResNetV2
SS_VGG16
SS_EfficientNetB0
SS_EfficientNetV2L
SS_DenseNet201
SS_ConvNeXtSmall
SS_EfficientNetB7
SS_InceptionV3
SS_EfficientNetB6
SS_EfficientNetB3
SS_EfficientNetV2M
SS_DenseNet121
SS_ResNet152
SS_EfficientNetV2B3
SS_ResNet101V2
SS_MobileNetV2
SS_ResNet152V2
SS_DenseNet169
SS_NASNetLarge
SS_NASNetMobile
SS_ConvNeXtLarge
SS_ResNet50
SS_EfficientNetV2S
SS_Xception
Starting execution of SS_EfficientNetV2B2... model nr.0 out of 38 models.
Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 0s 82ms/step
Monitoring stopped.
1 row(s) appended to Samples3.csv.
Appended to sample file:
['23:10:31:11:45:50', 'SS_EfficientNetV2B2', 4, 4, 2808.002, '1.92', '0.64', '1.07', '0.95', '0.93', 44.3645